# Dividend Income Analysis
Calculate historical yield for dividend income-generating stocks 

[2017 Dividend Aristrocrats List](https://www.simplysafedividends.com/dividend-aristocrats/)

[25+ Years of Dividend Increasing Stocks](http://www.dividend.com/dividend-stocks/25-year-dividend-increasing-stocks.php#stocks&sort_name=symbol&sort_order=asc&page=1)

In [4]:
library(xts)
library(zoo)
library(TTR)
library(quantmod)
setwd('/Users/jtan/Documents/Investment/Dividend-Income')

In [5]:
STOCK_LIST <- c('BDX')
START_DATE <- '2015-01-01'
END_DATE <- '2017-01-01'

start_date_lag <- function(start_date, num_year) {
    new_date <- as.POSIXlt(as.Date(start_date))
    new_date$year <- new_date$year - num_year
    new_date <- as.Date(new_date)
} 

In [8]:
for(i in 1:length(STOCK_LIST)) {

    cat('Processing', STOCK_LIST[i],'\n')

    # get pricing data
    data.price <- getSymbols(STOCK_LIST[i], from=START_DATE, to=END_DATE, auto.assign=FALSE)
    data.price <- data.frame(Date = index(data.price), data.price, row.names=NULL)
    colnames(data.price) <- c('Date','Open','High','Low','Close','Volume','Adjusted')
    data.price <- data.price[order(data.price$Date, decreasing=TRUE),]
    data.price$Symbol <- STOCK_LIST[i]

    # get dividend data
    temp <- getDividends(STOCK_LIST[i], from=start_date_lag(START_DATE, 1), to=END_DATE, auto.assign=FALSE)
    if(nrow(temp) == 0) {
        cat('Skipping', STOCK_LIST[i],'\n')
        next
    }
    data.div.quarter <- data.frame(Date = index(temp), temp, row.names=NULL)
    colnames(data.div.quarter) <- c('Date','Dividend')
    # reverse sort and do moving sum
    data.div.quarter <- data.div.quarter[order(data.div.quarter$Date, decreasing=TRUE),]  
    data.div.annual <- data.frame(matrix(NA, nrow=nrow(data.div.quarter)-3, ncol=2)) 
    colnames(data.div.annual) <- c('Date','Dividend')
    data.div.annual$Date <- head(data.div.quarter$Date, -3)
    data.div.annual$Dividend <- rollapply(data.div.quarter$Dividend, width = 4, by = 1, FUN = sum, align = "left")
    data.div.annual$Symbol <- STOCK_LIST[i]
    
    # merge/combine data
    data.combine <- merge(data.price, data.div.annual, by=c('Symbol','Date'), all=TRUE)
    # sort increasing to allow correct derivation of dividend
    data.combine <- data.combine[order(data.combine$Date),]
    if (is.na(data.combine$Dividend[1])) 
        data.combine$Dividend[1] <- tail(data.div.annual$Dividend, n=1)
    data.combine$Dividend <- na.locf(data.combine$Dividend)
    data.combine <- na.omit(data.combine)    
}

Processing BDX 


In [7]:
write.table(data.combine, file = 'Dividend.csv', sep = ",", col.names = TRUE, row.names = FALSE, quote = FALSE)